# MPPI (Model Predictive Path-Integral) Control

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
from matplotlib import patches
from matplotlib.animation import ArtistAnimation
from IPython import display

## Control Target : Vehicle

<img src="../media/pathtracking.png" width="500px">

In [2]:
class UnderwaterVehicle():
    def __init__(
        self,
        vehicle_length: float = 4.0,  # [m]
        vehicle_width: float = 3.0,   # [m]
        vehicle_height: float = 2.0,  # [m]
        max_lin_accel: float = 2.0,   # [m/s^2]
        max_ang_accel: float = 1.0,   # [rad/s^2]
        ref_path: np.ndarray = np.array([
            [-100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0,0,0,0,0,0],
            [ 100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0,0,0,0,0,0]
        ]),
        obstacle_spheres: np.ndarray = np.array([
            [-2.0, 1.0, 0.0, 1.0],
            [ 2.0, -1.0, 0.0, 1.0]
        ]),  # [obs_x, obs_y, obs_z, obs_radius]
        delta_t: float = 0.05,  # [s]
        visualize: bool = True,
    ) -> None:
        """
        Initialize underwater vehicle environment.
        State vector (12-dim):
            [x, y, z, roll, pitch, yaw, vx, vy, vz, wx, wy, wz]
        Control input (6-dim):
            [ax, ay, az, ax_ang, ay_ang, az_ang]
        """
        self.vehicle_length = vehicle_length
        self.vehicle_width = vehicle_width
        self.vehicle_height = vehicle_height
        self.max_lin_accel = max_lin_accel
        self.max_ang_accel = max_ang_accel
        self.delta_t = delta_t
        self.ref_path = ref_path  # expected to be (N, 12)
        self.obstacle_spheres = obstacle_spheres
        
        # Compute a collision radius (for a bounding sphere)
        self.vehicle_collision_radius = np.sqrt((vehicle_length/2)**2 + (vehicle_width/2)**2 + (vehicle_height/2)**2)

        # Visualization settings (we use a top-down x-y view)
        self.view_x_lim_min, self.view_x_lim_max = -20.0, 20.0
        self.view_y_lim_min, self.view_y_lim_max = -25.0, 25.0
        self.minimap_view_x_lim_min, self.minimap_view_x_lim_max = -40.0, 40.0
        self.minimap_view_y_lim_min, self.minimap_view_y_lim_max = -10.0, 40.0

        self.visualize_flag = visualize
        self.reset()

    def reset(
        self,
        init_state: np.ndarray = np.zeros(12),  # default: all zeros
    ) -> None:
        """Reset environment to the initial state."""
        self.state = init_state.copy()
        self.frames = []
        if self.visualize_flag:
            self.fig = plt.figure(figsize=(9,9))
            self.main_ax = plt.subplot2grid((3,4), (0,0), rowspan=3, colspan=3)
            self.minimap_ax = plt.subplot2grid((3,4), (0,3))
            # **Set equal aspect ratio**
            self.main_ax.set_aspect('equal')
            self.fig.tight_layout()

    def update(
        self,
        u: np.ndarray,  # 6-dim control input: [ax, ay, az, ax_ang, ay_ang, az_ang]
        delta_t: float = 0.0,
        optimal_traj: np.ndarray = np.empty(0),
        sampled_traj_list: np.ndarray = np.empty(0),
        append_frame: bool = True,
    ) -> None:
        """Update state using double-integrator dynamics in translation and rotation."""
        dt = self.delta_t if delta_t == 0.0 else delta_t
        # Unpack state: position, orientation, linear velocity, angular velocity
        pos    = self.state[0:3]
        orient = self.state[3:6]
        lin_vel = self.state[6:9]
        ang_vel = self.state[9:12]

        lin_acc = u[0:3]
        ang_acc = u[3:6]

        new_pos = pos + lin_vel*dt + 0.5*lin_acc*dt*dt
        new_lin_vel = lin_vel + lin_acc*dt
        new_orient = orient + ang_vel*dt + 0.5*ang_acc*dt*dt
        new_ang_vel = ang_vel + ang_acc*dt

        self.state = np.concatenate([new_pos, new_orient, new_lin_vel, new_ang_vel])

        if append_frame:
            self.append_frame(u, optimal_traj, sampled_traj_list)

    def get_state(self) -> np.ndarray:
        return self.state.copy()

def append_frame(self, steer: float, accel: float, optimal_traj: np.ndarray, sampled_traj_list: np.ndarray) -> list:
    """Draw a frame of the animation using the current state.

    The state is:
        [x, y, yaw, Vx, Vy, Vyaw, Ax, Ay, Ayaw]
    Control inputs:
        steer: angular acceleration command (for yaw) [rad/s^2]
        accel: linear acceleration command (for x direction) [m/s^2]
    """
    # Unpack the current state.
    x, y, yaw, Vx, Vy, Vyaw, Ax, Ay, Ayaw = self.state
    # Compute the speed from the x and y velocity components.
    speed = np.sqrt(Vx**2 + Vy**2)

    ### MAIN VIEW ###
    # Draw the vehicle shape as a rectangle.
    vw, vl = self.vehicle_w, self.vehicle_l
    # Define the rectangle (centered at (0,0)).
    vehicle_shape_x = [-0.5*vl, -0.5*vl, +0.5*vl, +0.5*vl, -0.5*vl]
    vehicle_shape_y = [-0.5*vw, +0.5*vw, +0.5*vw, -0.5*vw, -0.5*vw]
    # Rotate the shape by the current yaw (the vehicle is drawn at the center of the main view).
    rotated_vehicle_shape_x, rotated_vehicle_shape_y = self._affine_transform(vehicle_shape_x, vehicle_shape_y, yaw, [0, 0])
    frame = self.main_ax.plot(rotated_vehicle_shape_x, rotated_vehicle_shape_y, color='black', linewidth=2.0, zorder=3)

    # Draw the vehicle center as a small circle.
    vehicle_center = patches.Circle((0, 0), radius=vw/20.0, fc='white', ec='black', linewidth=2.0, zorder=6)
    frame += [self.main_ax.add_artist(vehicle_center)]

    # Draw the reference path (projected to x-y, relative to the current position).
    ref_path_x = self.ref_path[:, 0] - x
    ref_path_y = self.ref_path[:, 1] - y
    frame += self.main_ax.plot(ref_path_x, ref_path_y, color='black', linestyle="dashed", linewidth=1.5)

    # Draw an information text (showing the vehicle speed).
    text = "vehicle speed = {v:>+6.1f} [m/s]".format(v=speed)
    frame += [self.main_ax.text(0.5, 0.02, text, ha='center', transform=self.main_ax.transAxes, fontsize=14, fontfamily='monospace')]

    # Draw the predicted optimal trajectory from MPPI (projected to x-y relative to the current position).
    if optimal_traj.any():
        optimal_traj_x_offset = np.ravel(optimal_traj[:, 0]) - x
        optimal_traj_y_offset = np.ravel(optimal_traj[:, 1]) - y
        frame += self.main_ax.plot(optimal_traj_x_offset, optimal_traj_y_offset, color='#990099', linestyle="solid", linewidth=1.5, zorder=5)

    # Draw the sampled trajectories from MPPI.
    if sampled_traj_list.any():
        min_alpha_value = 0.25
        max_alpha_value = 0.35
        for idx, sampled_traj in enumerate(sampled_traj_list):
            # Make darker samples for better ones.
            alpha_value = (1.0 - (idx+1) / len(sampled_traj_list)) * (max_alpha_value - min_alpha_value) + min_alpha_value
            sampled_traj_x_offset = np.ravel(sampled_traj[:, 0]) - x
            sampled_traj_y_offset = np.ravel(sampled_traj[:, 1]) - y
            frame += self.main_ax.plot(sampled_traj_x_offset, sampled_traj_y_offset, color='gray', linestyle="solid", linewidth=0.2, zorder=4, alpha=alpha_value)

    ### MINI MAP VIEW ###
    frame += self.minimap_ax.plot(self.ref_path[:, 0], self.ref_path[:, 1], color='black', linestyle='dashed')
    rotated_vehicle_shape_x_minimap, rotated_vehicle_shape_y_minimap = self._affine_transform(vehicle_shape_x, vehicle_shape_y, yaw, [x, y])
    frame += self.minimap_ax.plot(rotated_vehicle_shape_x_minimap, rotated_vehicle_shape_y_minimap, color='black', linewidth=2.0, zorder=3)
    frame += self.minimap_ax.fill(rotated_vehicle_shape_x_minimap, rotated_vehicle_shape_y_minimap, color='white', zorder=2)

    ### CONTROL INPUT VIEWS ###
    # For the control inputs, we display angular acceleration (steer) and linear acceleration (accel).
    # Angular acceleration view:
    MAX_STEER = self.max_ang_abs
    PIE_RATE = 3.0/4.0
    PIE_STARTANGLE = 225  # degrees
    s_abs = np.abs(steer)
    if steer < 0.0:  # turning right
        steer_pie_obj, _ = self.steer_ax.pie(
            [MAX_STEER*PIE_RATE, s_abs*PIE_RATE, (MAX_STEER-s_abs)*PIE_RATE, 2*MAX_STEER*(1-PIE_RATE)],
            startangle=PIE_STARTANGLE, counterclock=False,
            colors=["lightgray", "black", "lightgray", "white"],
            wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
        )
    else:  # turning left
        steer_pie_obj, _ = self.steer_ax.pie(
            [(MAX_STEER-s_abs)*PIE_RATE, s_abs*PIE_RATE, MAX_STEER*PIE_RATE, 2*MAX_STEER*(1-PIE_RATE)],
            startangle=PIE_STARTANGLE, counterclock=False,
            colors=["lightgray", "black", "lightgray", "white"],
            wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
        )
    frame += steer_pie_obj
    frame += [self.steer_ax.text(0, -1, f"{np.rad2deg(steer):+.2f} " + r"$ \rm{[deg/s^2]}$", size=14, horizontalalignment='center', verticalalignment='center', fontfamily='monospace')]

    # Linear acceleration view:
    MAX_ACCEL = self.max_lin_abs
    PIE_RATE = 3.0/4.0
    PIE_STARTANGLE = 225
    a_abs = np.abs(accel)
    if accel > 0.0:
        accel_pie_obj, _ = self.accel_ax.pie(
            [MAX_ACCEL*PIE_RATE, a_abs*PIE_RATE, (MAX_ACCEL-a_abs)*PIE_RATE, 2*MAX_ACCEL*(1-PIE_RATE)],
            startangle=PIE_STARTANGLE, counterclock=False,
            colors=["lightgray", "black", "lightgray", "white"],
            wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
        )
    else:
        accel_pie_obj, _ = self.accel_ax.pie(
            [(MAX_ACCEL-a_abs)*PIE_RATE, a_abs*PIE_RATE, MAX_ACCEL*PIE_RATE, 2*MAX_ACCEL*(1-PIE_RATE)],
            startangle=PIE_STARTANGLE, counterclock=False,
            colors=["lightgray", "black", "lightgray", "white"],
            wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
        )
    frame += accel_pie_obj
    frame += [self.accel_ax.text(0, -1, f"{accel:+.2f} " + r"$ \rm{[m/s^2]}$", size=14, horizontalalignment='center', verticalalignment='center', fontfamily='monospace')]

    # Append the frame to the list of frames.
    self.frames.append(frame)

    def show_animation(self, interval_ms: int) -> None:
        ani = ArtistAnimation(self.fig, self.frames, interval=interval_ms)
        html = display.HTML(ani.to_jshtml())
        display.display(html)
        plt.close()


## Controller : MPPI Controller

### Note
The following MPPI implementation follows Algorithms 1 and 2 of the reference paper. 

### Reference
1. G. Williams et al. "Information-Theoretic Model Predictive Control: Theory and Applications to Autonomous Driving" 
    - URL : https://ieeexplore.ieee.org/document/8558663
    - PDF : https://arxiv.org/pdf/1707.02342.pdf

### Brief overview of MPPI algorithm
Here is a general process flow to calculate optimal input with mppi algorithm.

**[Step 1]** ramdomly sample input sequence

Mean input sequence $U$ and ramdomly sampled input sequence $V$ are defined as follows.  
Usually, optimal input sequence on the previous step is used as $U$. 
$$
    \begin{align}
        & (\mathbf{u}_0, \mathbf{u}_1, ... \mathbf{u}_{T-1}) = U \in \mathbb{R}^{m \times T}, \nonumber \\
        & (\mathbf{v}_0, \mathbf{v}_1, ... \mathbf{v}_{T-1}) = V \in \mathbb{R}^{m \times T}, \nonumber \\
        & \mathbf{v}_t = \mathbf{u}_t + \epsilon_t, \nonumber \\
        & \epsilon_t \sim \mathcal{N}(0, \Sigma).\nonumber 
    \end{align}
$$


**[Step 2]** predict future states and evaluate cost for each sample

We assume a discrete time, continuous state-action dynamical system as a control target.  
$\mathbf{x}$ is a system state, and $\mathbf{v}$ is a sampled control input.
$$
\begin{align}
\mathbf{x}_t  &\in \mathbb{R}^{n}, \nonumber \\
\mathbf{x}_{t+1} &= \mathbf{F}(\mathbf{x}_t, \mathbf{v}_t).\nonumber 
\end{align}
$$

Then costs (i.e. penalties to be minimized) for sampled sequences $S(V; \mathbf{x}_0)$ can be evaluated with following formulations.
$$
    \begin{align}
        & S(V; \mathbf{x}_0) = C(\mathcal{H}(V; \mathbf{x}_0)), \nonumber \\
        & C(\mathbf{x}_0, \mathbf{x}_1, ... \mathbf{x}_T) = \phi(\mathbf{x}_T) + \sum_{t=0}^{T-1}c(\mathbf{x}_t), \nonumber \\
        & \mathcal{H}(V; \mathbf{x}_0) = \left( \mathbf{x}_0, \mathbf{F}(\mathbf{x}_0, \mathbf{v}_0), \mathbf{F}(\mathbf{F}(\mathbf{x}_0, \mathbf{v}_0), \mathbf{v}_1), ... \right).\nonumber 
    \end{align}
$$

**[Step 3]** calculate weight for each sample sequence

Weight for a each sample sequence is derived on the basis of information theory.  
There are K sample sequences in total, represented with an index k.  
Good control sequence with small cost value get more weight, and vice versa.  

$$
\begin{align}
& w(V) = \frac{1}{\eta} \exp
\left( 
    -\frac{1}{\lambda}
    \left(
        S(V) + \lambda(1-\alpha) \sum^{T-1}_{t=0} \mathbf{u}_t^T \Sigma^{-1} (\mathbf{u}_t + \epsilon_t) - \rho
    \right)
\right) \nonumber \\
& \eta = 
\sum_{k=1}^K \exp
\left( 
    -\frac{1}{\lambda}
    \left(
        S(U + \mathcal{E}_k) + \lambda(1-\alpha) \sum^{T-1}_{t=0} \mathbf{u}_t^T \Sigma^{-1} (\mathbf{u}_t + \epsilon_t^k) - \rho
    \right)
\right)\nonumber \\
& \rho = 
\min_k 
\left( S(V_k) + \lambda(1-\alpha) \sum^{T-1}_{t=0} \mathbf{u}_t^T \Sigma^{-1} (\mathbf{u}_t + \epsilon_t^k) \right)\nonumber
\end{align}
$$

Note that $\rho$ is inserted into the formulation to avoid overflow errors during implementation.

**[Step 4]** get optimal control input sequence

Finally, optimal input trajectory for the next ($i+1$) step is given adding weighted sample sequences to the previous solution.

$$
\begin{align}
    \mathbf{u}_t^{i+1} % &= \mathbb{E}_{\mathbb{Q}_{\hat{U}, \Sigma}}[w(V)\mathbf{v}_t]
                 = u_t^i + \sum_{k=1}^K w(V_k) \epsilon_t^k \nonumber 
\end{align}
$$


In [3]:
class UnderwaterMPPIControllerForPathTracking():
    def __init__(
        self,
        delta_t: float = 0.1,
        max_lin_accel: float = 2.0,   # [m/s^2]
        max_ang_accel: float = 1.0,   # [rad/s^2]
        ref_path: np.ndarray = np.zeros((2, 12)),  # Each row: [pos(3), orient(3), lin_vel(3), ang_vel(3)]
        horizon_step_T: int = 20,
        number_of_samples_K: int = 500,
        param_exploration: float = 0.05,
        param_lambda: float = 100.0,
        param_alpha: float = 0.98,
        sigma: np.ndarray = np.diag([0.075, 0.075, 0.075, 0.05, 0.05, 0.05]), 
        stage_cost_weight: np.ndarray = np.ones(12),
        terminal_cost_weight: np.ndarray = np.ones(12),
        visualize_optimal_traj: bool = True,
        visualze_sampled_trajs: bool = True,
        obstacle_spheres: np.ndarray = np.array([
            [-2.0, 1.0, 0.0, 1.0],
            [ 2.0, -1.0, 0.0, 1.0]
        ]),  # [obs_x, obs_y, obs_z, obs_radius]
        collision_safety_margin_rate: float = 1.2,
        vehicle_collision_radius: float = 2.5,  # from the vehicle
    ) -> None:
        """
        Initialize the MPPI controller for underwater vehicle path tracking.
        State is 12-dim and control input is 6-dim.
        """
        self.dim_x = 12
        self.dim_u = 6
        self.T = horizon_step_T
        self.K = number_of_samples_K
        self.param_exploration = param_exploration
        self.param_lambda = param_lambda
        self.param_alpha = param_alpha
        self.param_gamma = self.param_lambda * (1.0 - self.param_alpha)
        self.Sigma = sigma
        self.stage_cost_weight = stage_cost_weight
        self.terminal_cost_weight = terminal_cost_weight
        self.visualize_optimal_traj = visualize_optimal_traj
        self.visualze_sampled_trajs = visualze_sampled_trajs

        self.delta_t = delta_t
        self.max_lin_accel = max_lin_accel
        self.max_ang_accel = max_ang_accel
        self.ref_path = ref_path
        self.obstacle_spheres = obstacle_spheres
        self.collision_safety_margin_rate = collision_safety_margin_rate
        self.vehicle_collision_radius = vehicle_collision_radius

        self.u_prev = np.zeros((self.T, self.dim_u))
        self.prev_waypoints_idx = 0

    def calc_control_input(self, observed_x: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """Calculate the optimal control input based on MPPI."""
        u = self.u_prev.copy()
        x0 = observed_x.copy()

        self._get_nearest_waypoint(x0[0], x0[1], x0[2], update_prev_idx=True)
        if self.prev_waypoints_idx >= self.ref_path.shape[0] - 1:
            print("[ERROR] Reached the end of the reference path.")
            raise IndexError

        S = np.zeros(self.K)
        epsilon = self._calc_epsilon(self.Sigma, self.K, self.T, self.dim_u)
        v = np.zeros((self.K, self.T, self.dim_u))

        for k in range(self.K):
            x = x0.copy()
            for t in range(1, self.T+1):
                if k < (1.0 - self.param_exploration)*self.K:
                    v[k, t-1] = u[t-1] + epsilon[k, t-1]
                else:
                    v[k, t-1] = epsilon[k, t-1]
                x = self._F(x, self._g(v[k, t-1]))
                S[k] += self._c(x) + self.param_gamma * u[t-1].T @ np.linalg.inv(self.Sigma) @ v[k, t-1]
            S[k] += self._phi(x)

        w = self._compute_weights(S)
        w_epsilon = np.zeros((self.T, self.dim_u))
        for t in range(self.T):
            for k in range(self.K):
                w_epsilon[t] += w[k] * epsilon[k, t]
        w_epsilon = self._moving_average_filter(w_epsilon, window_size=10)
        u += w_epsilon

        optimal_traj = np.zeros((self.T, self.dim_x))
        if self.visualize_optimal_traj:
            x = x0.copy()
            for t in range(self.T):
                x = self._F(x, self._g(u[t]))
                optimal_traj[t] = x

        sampled_traj_list = np.zeros((self.K, self.T, self.dim_x))
        sorted_idx = np.argsort(S)
        if self.visualze_sampled_trajs:
            for idx in sorted_idx:
                x = x0.copy()
                for t in range(self.T):
                    x = self._F(x, self._g(v[idx, t]))
                    sampled_traj_list[idx, t] = x

        self.u_prev[:-1] = u[1:]
        self.u_prev[-1] = u[-1]

        return u[0], u, optimal_traj, sampled_traj_list

    def _calc_epsilon(self, sigma: np.ndarray, size_sample: int, size_time_step: int, size_dim_u: int) -> np.ndarray:
        if sigma.shape[0] != sigma.shape[1] or sigma.shape[0] != size_dim_u or size_dim_u < 1:
            raise ValueError("sigma must be a square matrix with the size of size_dim_u.")
        mu = np.zeros(size_dim_u)
        epsilon = np.random.multivariate_normal(mu, sigma, (size_sample, size_time_step))
        return epsilon

    def _g(self, v: np.ndarray) -> np.ndarray:
        # Clip linear acceleration inputs
        v[0] = np.clip(v[0], -self.max_lin_accel, self.max_lin_accel)
        v[1] = np.clip(v[1], -self.max_lin_accel, self.max_lin_accel)
        v[2] = np.clip(v[2], -self.max_lin_accel, self.max_lin_accel)
        # Clip angular acceleration inputs
        v[3] = np.clip(v[3], -self.max_ang_accel, self.max_ang_accel)
        v[4] = np.clip(v[4], -self.max_ang_accel, self.max_ang_accel)
        v[5] = np.clip(v[5], -self.max_ang_accel, self.max_ang_accel)
        return v

    def _c(self, x_t: np.ndarray) -> float:
        # Compute the stage cost as the weighted squared error against the nearest waypoint.
        pos = x_t[0:3]
        _, ref_x, ref_y, ref_z, ref_roll, ref_pitch, ref_yaw, ref_vx, ref_vy, ref_vz, ref_wx, ref_wy, ref_wz = self._get_nearest_waypoint(pos[0], pos[1], pos[2])
        ref_state = np.array([ref_x, ref_y, ref_z, ref_roll, ref_pitch, ref_yaw, ref_vx, ref_vy, ref_vz, ref_wx, ref_wy, ref_wz])
        error = x_t - ref_state
        stage_cost = np.sum(self.stage_cost_weight * (error**2))
        stage_cost += self._is_collided(x_t) * 1.0e10
        return stage_cost

    def _phi(self, x_T: np.ndarray) -> float:
        pos = x_T[0:3]
        _, ref_x, ref_y, ref_z, ref_roll, ref_pitch, ref_yaw, ref_vx, ref_vy, ref_vz, ref_wx, ref_wy, ref_wz = self._get_nearest_waypoint(pos[0], pos[1], pos[2])
        ref_state = np.array([ref_x, ref_y, ref_z, ref_roll, ref_pitch, ref_yaw, ref_vx, ref_vy, ref_vz, ref_wx, ref_wy, ref_wz])
        terminal_cost = np.sum(self.terminal_cost_weight * ((x_T - ref_state)**2))
        terminal_cost += self._is_collided(x_T) * 1.0e10
        return terminal_cost

    def _is_collided(self, x_t: np.ndarray) -> float:
        pos = x_t[0:3]
        for obs in self.obstacle_spheres:
            obs_center = np.array(obs[0:3])
            obs_r = obs[3]
            if np.linalg.norm(pos - obs_center) < (self.vehicle_collision_radius * self.collision_safety_margin_rate + obs_r):
                return 1.0
        return 0.0

    def _F(self, x_t: np.ndarray, u: np.ndarray) -> np.ndarray:
        """Simulate one step forward using double-integrator dynamics."""
        dt = self.delta_t
        pos    = x_t[0:3]
        orient = x_t[3:6]
        lin_vel = x_t[6:9]
        ang_vel = x_t[9:12]

        lin_acc = u[0:3]
        ang_acc = u[3:6]

        new_pos = pos + lin_vel*dt + 0.5*lin_acc*dt*dt
        new_lin_vel = lin_vel + lin_acc*dt
        new_orient = orient + ang_vel*dt + 0.5*ang_acc*dt*dt
        new_ang_vel = ang_vel + ang_acc*dt

        return np.concatenate([new_pos, new_orient, new_lin_vel, new_ang_vel])

    def _compute_weights(self, S: np.ndarray) -> np.ndarray:
        w = np.zeros(self.K)
        rho = S.min()
        eta = np.sum(np.exp((-1.0/self.param_lambda) * (S - rho)))
        for k in range(self.K):
            w[k] = (1.0 / eta) * np.exp((-1.0/self.param_lambda) * (S[k] - rho))
        return w

    def _moving_average_filter(self, xx: np.ndarray, window_size: int) -> np.ndarray:
        b = np.ones(window_size) / window_size
        dim = xx.shape[1]
        xx_mean = np.zeros(xx.shape)
        for d in range(dim):
            xx_mean[:, d] = np.convolve(xx[:, d], b, mode="same")
            n_conv = math.ceil(window_size/2)
            xx_mean[0, d] *= window_size / n_conv
            for i in range(1, n_conv):
                xx_mean[i, d] *= window_size / (i + n_conv)
                xx_mean[-i, d] *= window_size / (i + n_conv - (window_size % 2))
        return xx_mean

    def _get_nearest_waypoint(self, x: float, y: float, z: float, update_prev_idx: bool = False):
        SEARCH_IDX_LEN = 200
        prev_idx = self.prev_waypoints_idx
        # Search only over position (x,y,z)
        dx = self.ref_path[prev_idx:(prev_idx + SEARCH_IDX_LEN), 0] - x
        dy = self.ref_path[prev_idx:(prev_idx + SEARCH_IDX_LEN), 1] - y
        dz = self.ref_path[prev_idx:(prev_idx + SEARCH_IDX_LEN), 2] - z
        d = dx**2 + dy**2 + dz**2
        min_idx_offset = np.argmin(d)
        nearest_idx = prev_idx + min_idx_offset
        waypoint = self.ref_path[nearest_idx]
        if update_prev_idx:
            self.prev_waypoints_idx = nearest_idx
        return (nearest_idx, *waypoint)


In [ ]:
# Number of waypoints for the underwater path
N = 100
t = np.linspace(0, 4*np.pi, N)

# Generate a helical path in 3D
x = 10 * np.cos(t)         # x coordinate
y = 10 * np.sin(t)         # y coordinate
z = np.linspace(0, 20, N)    # z coordinate (linearly increasing)

# For simplicity, set roll and pitch to zero, yaw varying with t.
roll  = np.zeros(N)
pitch = np.zeros(N)
yaw   = t

# Assume constant velocities along the path (dummy values)
# Here, we approximate vx and vy based on the derivative of the helix
vx = -10 * np.sin(t) * (4*np.pi/(N-1))
vy =  10 * np.cos(t) * (4*np.pi/(N-1))
vz = np.full(N, 20/(N-1))  # constant z velocity

# Assume zero angular velocities for simplicity
wx = np.zeros(N)
wy = np.zeros(N)
wz = np.zeros(N)

# Stack all columns into a single array. Each row is a waypoint:
# [x, y, z, roll, pitch, yaw, vx, vy, vz, wx, wy, wz]
path = np.stack([x, y, z, roll, pitch, yaw, vx, vy, vz, wx, wy, wz], axis=1)

# Header for the CSV file
header = "x,y,z,roll,pitch,yaw,vx,vy,vz,wx,wy,wz"

# Save the array to underwater_path.csv
np.savetxt("../data/underwater_path.csv", path, delimiter=",", header=header, comments="", fmt="%.4f")

print("underwater_path.csv has been generated.")


## Simulation

In [ ]:
# Simulation settings
delta_t = 0.05  # [sec]
sim_steps = 150
print(f"[INFO] delta_t : {delta_t:.2f}[s] , sim_steps : {sim_steps}[steps], total_sim_time : {delta_t*sim_steps:.2f}[s]")

# Define obstacle spheres: each row is [x, y, z, radius]
OBSTACLE_SPHERES = np.array([
    [8.0, 5.0, 0.0, 4.0],
    [18.0, -5.0, 0.0, 4.0],
])

# Load reference path from CSV.
# (Assumes the file has 12 columns: [x, y, z, roll, pitch, yaw, vx, vy, vz, wx, wy, wz])
ref_path = np.genfromtxt('../data/underwater_path.csv', delimiter=',', skip_header=1)
plt.figure()
plt.title("Reference Path (Top-Down View)")
plt.plot(ref_path[:,0], ref_path[:,1])
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.show()

# Initialize the underwater vehicle.
# **Start the vehicle on the path by using the first waypoint as the initial state.**
vehicle = UnderwaterVehicle(
    vehicle_length=4.0,
    vehicle_width=3.0,
    vehicle_height=2.0,
    max_lin_accel=2.0,
    max_ang_accel=1.0,
    ref_path=ref_path,
    obstacle_spheres=OBSTACLE_SPHERES,
    delta_t=delta_t,
)
vehicle.reset(init_state=ref_path[0])  # start at the first waypoint

# Initialize the MPPI controller for the underwater vehicle.
mppi = UnderwaterMPPIControllerForPathTracking(
    delta_t=delta_t*2.0,
    max_lin_accel=2.0,
    max_ang_accel=1.0,
    ref_path=ref_path,
    horizon_step_T=20,
    number_of_samples_K=500,
    param_exploration=0.05,
    param_lambda=100.0,
    param_alpha=0.98,
    sigma=np.diag([0.075, 0.075, 0.075, 0.05, 0.05, 0.05]),
    stage_cost_weight=np.ones(12),
    terminal_cost_weight=np.ones(12),
    visualze_sampled_trajs=True,
    obstacle_spheres=OBSTACLE_SPHERES,
    collision_safety_margin_rate=1.2,
    vehicle_collision_radius=vehicle.vehicle_collision_radius,
)

# Simulation loop
for i in range(sim_steps):
    current_state = vehicle.get_state()
    try:
        optimal_input, optimal_input_sequence, optimal_traj, sampled_traj_list = mppi.calc_control_input(current_state)
    except IndexError as e:
        print("[ERROR] IndexError detected. Terminate simulation.")
        break

    print(f"Time: {i*delta_t:>2.2f}[s], "
          f"Pos=({current_state[0]:+.2f},{current_state[1]:+.2f},{current_state[2]:+.2f}), "
          f"Orient=({current_state[3]:+.2f},{current_state[4]:+.2f},{current_state[5]:+.2f}), "
          f"LinVel=({current_state[6]:+.2f},{current_state[7]:+.2f},{current_state[8]:+.2f}), "
          f"AngVel=({current_state[9]:+.2f},{current_state[10]:+.2f},{current_state[11]:+.2f}), "
          f"Control=({optimal_input[0]:+.2f},{optimal_input[1]:+.2f},{optimal_input[2]:+.2f},"
          f"{optimal_input[3]:+.2f},{optimal_input[4]:+.2f},{optimal_input[5]:+.2f})")
    
    # Pass the 3D positions (first three state components) for trajectory visualization.
    vehicle.update(u=optimal_input, delta_t=delta_t, optimal_traj=optimal_traj[:, 0:3], sampled_traj_list=sampled_traj_list[:, :, 0:3])

vehicle.show_animation(interval_ms=int(delta_t * 1000))
